In [1]:
import zipfile

filepath = r'C:\Users\kwon\project\project\dacon\resume\data.zip'
folder = r'C:\Users\kwon\project\project\dacon\resume\data'

with zipfile.ZipFile(filepath, 'r') as zip_ref :
    zip_ref.extractall(folder)

In [2]:
import pandas as pd
import os

data = {}
data_path = r'C:\Users\kwon\project\project\dacon\resume\data'
for fname in os.listdir(data_path):
    if fname.endswith('.csv'):
        file_path = os.path.join(data_path, fname)
        data[fname.rsplit('.', 1)[0]] = pd.read_csv(file_path)


### 데이터 쪼개기

In [11]:
apply = data['apply_train']

In [12]:
from sklearn.model_selection import train_test_split

apply_train_df, apply_test_df = train_test_split(apply, test_size= 0.25)

### CSR 매트릭스

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)

In [14]:
from scipy.sparse import csr_matrix
user_item = csr_matrix(user_item_matrix)

In [21]:
print(user_item)

  (0, 4524)	1
  (0, 5197)	1
  (1, 1100)	1
  (1, 1727)	1
  (1, 1956)	1
  (1, 2942)	1
  (1, 4576)	1
  (1, 5459)	1
  (1, 6006)	1
  (2, 1457)	1
  (2, 3293)	1
  (2, 4905)	1
  (3, 3)	1
  (3, 311)	1
  (3, 1065)	1
  (3, 1508)	1
  (3, 1744)	1
  (3, 1822)	1
  (3, 1999)	1
  (3, 2351)	1
  (3, 2895)	1
  (3, 3917)	1
  (3, 4697)	1
  (3, 4878)	1
  (3, 5328)	1
  :	:
  (8328, 2264)	1
  (8328, 4091)	1
  (8328, 6620)	1
  (8329, 881)	1
  (8329, 3493)	1
  (8329, 4354)	1
  (8330, 889)	1
  (8330, 1550)	1
  (8330, 3736)	1
  (8331, 3952)	1
  (8332, 886)	1
  (8332, 5455)	1
  (8333, 1995)	1
  (8334, 2166)	1
  (8334, 3931)	1
  (8335, 1467)	1
  (8335, 2783)	1
  (8335, 2984)	1
  (8335, 3829)	1
  (8335, 4027)	1
  (8335, 4185)	1
  (8336, 3145)	1
  (8337, 3625)	1
  (8337, 3735)	1
  (8337, 6269)	1


In [22]:
import implicit
import numpy as np

model = implicit.als.AlternatingLeastSquares(factors=5, iterations=20)
model.fit(user_item.T)
user_id = 10

model.recommend(user_id, user_item.T[user_id], N = 10, filter_already_liked_items=False)

c:\Users\kwon\project\project\dacon\resume\venv\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.001001119613647461 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

(array([6654, 7813, 5519, 3551, 2251, 2721, 3677,    5,  490, 5964]),
 array([0.00237361, 0.00171669, 0.00160679, 0.00157252, 0.00126443,
        0.00122785, 0.00121791, 0.00117051, 0.00116878, 0.00113847],
       dtype=float32))

In [27]:
item_id, _ = model.recommend(user_id, user_item.T[10], N=5, filter_already_liked_items=False)

In [28]:
item_id, _

(array([1004,  120, 6742, 4299, 6169]),
 array([0.0045308 , 0.00403419, 0.00353407, 0.00352699, 0.00323935],
       dtype=float32))

In [24]:
model = implicit.als.AlternatingLeastSquares(factors=5, iterations=20)
model.fit(user_item.T)

recommendations = []

for user_id in range(1, 10):
    # 각 사용자에 대한 5개의 추천 항목 얻기
    recs = model.recommend(user_id, user_item.T[user_id], N=5, filter_already_liked_items=False)
    
    # 추천 항목을 원하는 형식으로 변환하여 recommendations 리스트에 추가
    for item_id, _ in recs:
        recommendations.append(f"U{str(user_id).zfill(5)},R{str(item_id).zfill(5)}")

# 결과 출력
for rec in recommendations:
    print(rec)

c:\Users\kwon\project\project\dacon\resume\venv\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)